## Import modules

In [1]:
# def load_data(loc:str, index:list):
#     df = pandas.read_csv(f'data/{loc}_solar19.csv')
#     df['idx'] = index
#     df = df.set_index('idx')
#     return df
# idx = [(i,j,k) for i,j,k in product(range(1), range(365), range(24))]
# weather_ny = load_data('ny', idx)[['DNI', 'Wind Speed']]
# weather_ny = weather_ny.rename(columns= {'DNI': 'dni', 'Wind Speed': 'wind_speed'})
# weather_sd = load_data('sd', idx)[['dni', 'wind_speed']]
# weather_ho = load_data('ho', idx)[['dni', 'wind_speed']]""
# demand_sd = pandas.read_excel('data/HistoricalEMSHourlyLoad-2019.xlsx', index_col= 0)[['SDGE']]
# demand_ho = pandas.read_excel('data/Native_Load_2019.xlsx')[['COAST']]
# demand_ny = DataFrame(pandas.read_csv('data/NYC_load.csv')['Load']) #from Will and Doga
# ng_price = make_henry_price_df(
#     file_name='data/Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2020)
# ng_price = ng_price.drop(columns= 'scales')
# ng_price = remove_outliers(ng_price, sd_cuttoff = 3)
# ng_price = pandas.concat([ng_price])

In [2]:
import sys



sys.path.append('src')

In [3]:
from pandas import DataFrame
from energiapy.components import *
from energiapy.blocks.scenario import Scenario
from energiapy.parameters.designators.mode import X
from energiapy.parameters.designators.incidental import I

In [4]:
from dataclasses import dataclass, fields

In [5]:
Player.inputs()

['has', 'needs']

In [6]:
from energiapy._core._aliases._is_input import IsInput

In [7]:
s = Scenario(default=True)

In [8]:
# s.h2 = Horizon({'days': 2, 'hours': 12})
s.hor = Horizon(discretizations=[2, 12])
s.scales
s.net = Network(['madgaon', 'ponje', 'cacoda'])
s.nodes
s.alink = Linkage(source=s.madgaon, sink=s.ponje, bi=True, distance=50)
s.blink = Linkage(source=s.ponje, sink=s.cacoda, bi=False, distance=80)
s.clink = Linkage(source=s.cacoda, sink=s.madgaon, bi=True, distance=100)
s.dlink = Linkage(source=s.madgaon, sink=s.ponje, bi=False, distance=200)

c:\Users\cacodcar\energiapy\src\energiapy\blocks\scenario.py:243: UserWarning: 
A hrz_def object was already defined.
Overwriting.
This should not cause any modeling issues.
Check Scenario defaults if unintended.

  warn(
c:\Users\cacodcar\energiapy\src\energiapy\blocks\scenario.py:243: UserWarning: 
A ntw_def object was already defined.
Overwriting.
This should not cause any modeling issues.
Check Scenario defaults if unintended.

  warn(


In [9]:
s.system.scales

[t0, t1, t2]

In [10]:
s.scales

[t0, t1, t2]

In [11]:
a = DataFrame({'a': list(range(2))})
b = DataFrame({'b': list(range(24))})

In [12]:
s.csh = Cash(
    spend={
        s.madgaon: True,
        s.ponje: {s.t1: (2, a)},
        s.cacoda: 300,
        s.network: {s.t2: [5, 60]},
    }
)

c:\Users\cacodcar\energiapy\src\energiapy\blocks\scenario.py:243: UserWarning: 
A csh_def object was already defined.
Overwriting.
This should not cause any modeling issues.
Check Scenario defaults if unintended.

  warn(


In [13]:
s.csh.constraints

[BindLB[Spend(csh, net, t2)],
 BindUB[Spend(csh, madgaon, t0)],
 BindUB[Spend(csh, net, t2)],
 Bind[Spend(csh, cacoda, t0)],
 Bind[Spend(csh, ponje, t1)]]

In [14]:
s.csh.variables

[Spend(csh, cacoda, t0),
 Spend(csh, madgaon, t0),
 Spend(csh, net, t2),
 Spend(csh, ponje, t1)]

In [15]:
s.data.__dict__

{'name': 'Data|:s:|',
 '_initial_state': {'name': ':s:'},
 'lnd_def': Data|lnd_def|,
 'gwp': Data|gwp|,
 'ap': Data|ap|,
 'epm': Data|epm|,
 'epf': Data|epf|,
 'ept': Data|ept|,
 'pocp': Data|pocp|,
 'odp': Data|odp|,
 'adpmn': Data|adpmn|,
 'adpmt': Data|adpmt|,
 'adpf': Data|adpf|,
 'wdp': Data|wdp|,
 'dm': Data|dm|,
 'market': Data|market|,
 'consumer': Data|consumer|,
 'earth': Data|earth|,
 'csh': Data|csh|}

In [16]:
s.csh.data.__dict__

{'component': csh,
 'name': 'Data|csh|',
 'spend': [M(csh, madgaon, t0),
  300(csh, cacoda, t0),
  Th(csh, ponje, t1),
  [5(csh, net, t2):L, 60(csh, net, t2):U]]}

In [17]:
s.csh.parameters

[SpendBnd|300(csh, cacoda, t0)|,
 SpendBnd|5(csh, net, t2):L|,
 SpendBnd|60(csh, net, t2):U|,
 SpendBnd|M(csh, madgaon, t0)|,
 SpendBnd|Th(csh, ponje, t1)|]

In [18]:
s.lnd = Land(use={s.t2: 100, s.ponje: {s.t0: b}, s.cacoda: a}, cost=20)

c:\Users\cacodcar\energiapy\src\energiapy\blocks\scenario.py:243: UserWarning: 
A lnd_def object was already defined.
Overwriting.
This should not cause any modeling issues.
Check Scenario defaults if unintended.

  warn(
c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:160: UserWarning: lnd.use:Inconsistent temporal scale for ponje at t0. Updating to t2
  warn(


In [19]:
# s.lnd.parameters
# s.lnd.constraints
# s.lnd.variables

In [20]:
s.H2 = Resource(
    sell={s.t0: [0, 4], s.t1: [a, 34], s.t2: (a, b)},
    sell_price={
        s.madgaon: True,
        s.ponje: {s.t1: (2, b)},
        s.cacoda: 300,
        s.net: {s.t0: [5, 60]},
    },
)


s.Solar = Resource(buy=True, basis='MW', label='Solar Power')


s.Wind = Resource(buy=a, basis='MW', label='Wind Power')


s.Power = Resource(basis='MW', label='Power generated')


s.Uranium = Resource(
    buy=DataFrame({'a': [i for i in range(24)]}),
    buy_price=42.70 / (250 / 2),
    basis='kg',
    label='Uranium',
)


s.H2_L = Resource(sell=(0, 23), basis='tons', label='Hydrogen')


s.CO2_AQoff = Resource(basis='tons', label='Carbon dioxide - sequestered')


s.H2O = Resource(buy=(20, 50), buy_price=b, basis='tons', label='Water')


s.CH4 = Resource(buy=[20, 40], buy_price=20, basis='tons', label='Natural gas')


s.CO2 = Resource(basis='tons', label='Carbon dioxide', block='Resource')


s.CO2_Vent = Resource(
    basis='tons', label='Carbon dioxide - Vented', sell_price=(2, 20))


s.O2 = Resource(sell=[20, True], basis='tons', label='Oxygen')


s.CO2_DAC = Resource(basis='tons', label='Carbon dioxide - captured')
s.Power = Resource(buy=[0, a], basis='MW', label='Power generated')

c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:198: UserWarning: H2.sell_price:Inconsistent temporal scale for ponje at t1. Updating to t2
  warn(
c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:198: UserWarning: Power.buy:Inconsistent temporal scale for net at t0. Updating to t1
  warn(


In [21]:
s.LiR = Material(
    emission={s.gwp: 1.484},
    use={s.ponje: 1.5},
    basis='kg',
    label='Lithium Reserves',
    citation='Nelson Bunyui Manjong (2021)',
)


# LiB = Material(name='LiB', gwp=0.031, toxicity=793, basis='kg', label='Brine-based Lithium',


#                citation='Nelson Bunyui Manjong (2021)')


# Monocrystalline = Material(name='Mc', gwp=20, resource_cons={H2O: 1.5}, toxicity=40.74, basis='kg', label='Mono',


#                            citation='Environmental impact assessment of monocrystalline silicon solar photovoltaic cell production: a case study in China (toxicity)')


# Polycrystalline = Material(name='Pc', gwp=26, resource_cons={


#                            H2O: 1}, toxicity=15, basis='kg', label='Poly', citation='Environmental Economic Impact Assessments of a ... - MDPI (gwp and toxicity)')


# CdTE = Material(name='CdTe', gwp=40, resource_cons={H2O: 0.25}, toxicity=5, basis='kg', label='C',


#                 citation='Investigation of life cycle CO2 emissions of the polycrystalline and cadmium telluride PV panels')

In [22]:
s.WF = Process(
    conversion={s.Power: {X(0): {s.Wind: -1}, X(1): {s.Wind: -1}}},
    produce=[True, b],
    capex={1462000, I(202233)},
    opex={4953, I(70)},
    capacity=1000,
    emission={s.gwp: 50, s.odp: 2900},
    material={s.LiR: 1.5},
    land=30,
    label='Wind mill array',
    basis='MW',
)


s.PV = Process(
    sell_price={s.Power: 30},
    credit={s.Power: DataFrame({'a': [2]})},
    conversion={s.Power: {X(1): {s.Solar: -1.2}, X(2): {s.Solar: -1.5}}},
    capex={X(0): 1333, X(1): 1444, X(2): 1555},
    opex=22623,
    capacity={X(0): 3000, X(1): 4000, X(2): 5000},
    label='Solar PV',
    basis='MW',
)


s.SMRH = Process(
    land=50,
    conversion={
        s.H2: {
            s.Power: -1.11,
            s.CH4: -3.76,
            s.H2O: -23.7,
            s.CO2_Vent: 1.03,
            s.CO2: 9.332,
        }
    },
    capex={
        2520000,
    },
    opex={51.5, I(945000)},
    capacity=1000,
    label='Steam methane reforming + CCUS',
)

s.NGCC = Process(
    buy_price={s.CH4: 4, s.H2O: 20},
    conversion={
        s.Power: {
            s.CH4: -0.108,
            s.H2O: -10,
            s.CO2_Vent: 0.297 * 0.05,
            s.CO2: 0.297 * 0.95,
        }
    },
    capex=2158928,
    opex={4090, I(53320)},
    capacity=[1, 100],
    label='NGCC + 95% CC',
)
s.SMR = Process(
    capex=2400,
    opex={0.03, I(800)},
    conversion={s.H2: {s.Power: -1.11, s.CH4: -
                       3.76, s.H2O: -23.7, s.CO2_Vent: 9.4979}},
    capacity=1000,
    label='Steam methane reforming',
)
s.H2FC = Process(
    buy_price={s.H2: 2},
    sell={s.Power: 20},
    conversion={s.Power: {s.H2: -0.050}},
    capex=1.6 * 10**6,
    opex=3.5,
    capacity=[1000, 2000],
    label='hydrogen fuel cell',
)
s.DAC = Process(
    capex=0.02536,
    opex=0.634,
    conversion={s.CO2_DAC: {s.Power: -0.193, s.H2O: -4.048}},
    capacity=1000,
    label='Direct air capture',
)
# s.PSH = Process(conversion = {s.Power: 0.6}, capex = 3924781, fopex= 17820, vopex = 512.5, store = 10000, capacity=1000, label='Pumped storage hydropower', basis = 'MW')
s.ASMR = Process(
    buy={s.Uranium: 40},
    conversion={s.Power: {s.Uranium: -4.17 * 10 ** (-5), s.H2O: -3.364}},
    capex=7988951,
    opex=I(0.04 * 0.730),
    capacity=1000,
    label='Small modular reactors (SMRs)',
)
s.AWE = Process(
    land=(0, 20),
    conversion={s.H2: {s.Power: -1, s.O2: 0.7632, s.H2O: -0.1753}},
    capex={1.1 * 10**6, I(20)},
    opex=I(16918),
    capacity=True,
    label='Alkaline water electrolysis (AWE)',
    citation='Demirhan et al. 2018 AIChE paper',
)

c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:198: UserWarning: WF.operate:Inconsistent temporal scale for net at t0. Updating to t2
  warn(


In [23]:
Process.inputs()

['capacity',
 'operate',
 'capex',
 'opex',
 'emission',
 'land',
 'buy',
 'sell',
 'buy_price',
 'sell_price',
 'credit',
 'penalty',
 'material']

In [24]:
Storage.inputs()

['capacity',
 'operate',
 'capex',
 'opex',
 'emission',
 'land',
 'material',
 'loss']

In [25]:
s.LiI = Storage(capacity=[2000], inventory=s.Power)
s.LiI2 = Storage(capacity=[2000], inventory={s.Power: 0.6})
s.H2Stg = Storage(
    capacity=[2000], inventory={s.H2: {X(0): {s.Power: 0.8}, X(1): {s.Power: 0.9}}}
)

{Power: 0.8}
{Power: 0.9}


In [29]:
s.H2Stg.balance_c

{x(H2Stg_c,conv,(0)): {H2Stg_H2: 1, Power: -1.25, H2: -1},
 x(H2Stg_c,conv,(1)): {H2Stg_H2: 1, Power: -1.1111111111111112, H2: -1}}

In [28]:
s.LiI.balance_d

{Power: 1, LiI_Power: -1}

In [26]:
# s.H2Stg.cin.balance
s.H2Stg_c.conversion.balance

{x(H2Stg_c,conv,(0)): {H2Stg_H2: 1, Power: -1.25, H2: -1},
 x(H2Stg_c,conv,(1)): {H2Stg_H2: 1, Power: -1.1111111111111112, H2: -1}}

In [34]:
s.LiI.inventory.conversion_c.

SyntaxError: invalid syntax (1167265362.py, line 1)

In [29]:
s.LiI.eqns()

Capacity(LiI, net, t0)>=CapBnd|0(LiI, net, t0):L|
Capacity(LiI, net, t0)<=CapBnd|2000(LiI, net, t0):U|


In [30]:
s.LiI_c.eqns()

Capacity(LiI_c, net, t0)<=CapBnd|M(LiI_c, net, t0)|


In [ ]:
s.LiI2_d.conversion.balance
s.H2Stg_d.conversion.balance

{H2: 1, H2Stg_H2: 1}

In [29]:
s.resources

[CH4,
 CO2,
 CO2_AQoff,
 CO2_DAC,
 CO2_Vent,
 H2,
 H2O,
 H2Stg_H2,
 H2_L,
 LiI2_Power,
 LiI_Power,
 O2,
 Power,
 Solar,
 Uranium,
 Wind]

In [30]:
isinstance(s.LiI_Power, Resource)

True

In [37]:
from energiapy.components._base._component import _Component
from energiapy.components._base._defined import _Defined

In [38]:
isinstance(s.H2, _Defined)

True

In [33]:
s.storages

[H2Stg, LiI, LiI2]

In [32]:
s.processes

[ASMR,
 AWE,
 DAC,
 H2FC,
 H2Stg_c,
 H2Stg_d,
 LiI2_c,
 LiI2_d,
 LiI_c,
 LiI_d,
 NGCC,
 PV,
 SMR,
 SMRH,
 WF]

In [ ]:
s.LiI_c.conversion.__dict__

{'conversion': {'resource_str': {Power: -1}},
 'process': LiI_c,
 'base': 'resource_str',
 'modes': None,
 'n_modes': 1,
 'sold': ['resource_str'],
 'bought': [Power],
 'balance': {'resource_str': 1, Power: -1},
 'involve': ['resource_str', Power],
 'name': 'Conv(resource_str,LiI_c)'}

In [ ]:
s.WF.operate

[[M(WF, net, t2):L, DtSt(WF, net, t2):U]]

In [25]:
s.data.__dict__

{'name': 'Data|:s:|',
 '_initial_state': {'name': ':s:'},
 'gwp': Data|gwp|,
 'ap': Data|ap|,
 'epm': Data|epm|,
 'epf': Data|epf|,
 'ept': Data|ept|,
 'pocp': Data|pocp|,
 'odp': Data|odp|,
 'adpmn': Data|adpmn|,
 'adpmt': Data|adpmt|,
 'adpf': Data|adpf|,
 'wdp': Data|wdp|,
 'dm': Data|dm|,
 'market': Data|market|,
 'consumer': Data|consumer|,
 'earth': Data|earth|,
 'csh': Data|csh|,
 'lnd': Data|lnd|,
 'H2': Data|H2|,
 'Solar': Data|Solar|,
 'Wind': Data|Wind|,
 'Power': Data|Power|,
 'Uranium': Data|Uranium|,
 'H2_L': Data|H2_L|,
 'CO2_AQoff': Data|CO2_AQoff|,
 'H2O': Data|H2O|,
 'CH4': Data|CH4|,
 'CO2': Data|CO2|,
 'CO2_Vent': Data|CO2_Vent|,
 'O2': Data|O2|,
 'CO2_DAC': Data|CO2_DAC|,
 'LiR': Data|LiR|,
 'WF': Data|WF|,
 'PV': Data|PV|,
 'SMRH': Data|SMRH|,
 'NGCC': Data|NGCC|,
 'SMR': Data|SMR|,
 'H2FC': Data|H2FC|,
 'DAC': Data|DAC|,
 'ASMR': Data|ASMR|,
 'AWE': Data|AWE|}

In [26]:
s.SMR.constants

[0.03(csh, SMR, net, t0),
 800(csh, SMR, net, t0):i,
 1000(SMR, net, t0),
 2400(csh, SMR, net, t0)]

In [27]:
s.SMR.eqns()

Capacity(SMR, net, t0)==CapBnd|1000(SMR, net, t0)|
ExpCap(csh, SMR, net, t0)==CapExp|2400(csh, SMR, net, t0)|*Capacity(SMR, net, t0)
ExpOp(csh, SMR, net, t0)==OpExp|0.03(csh, SMR, net, t0)|*Operate(SMR, net, t0)
ExpOpI(csh, SMR, net, t0)==OpExpI|800(csh, SMR, net, t0):i|


In [28]:
s.eqns()

-----------gwp Constraints -----------

-----------ap Constraints -----------

-----------epm Constraints -----------

-----------epf Constraints -----------

-----------ept Constraints -----------

-----------pocp Constraints -----------

-----------odp Constraints -----------

-----------adpmn Constraints -----------

-----------adpmt Constraints -----------

-----------adpf Constraints -----------

-----------wdp Constraints -----------

-----------dm Constraints -----------

-----------market Constraints -----------

-----------consumer Constraints -----------

-----------earth Constraints -----------

-----------csh Constraints -----------
Spend(csh, net, t2)>=SpendBnd|5(csh, net, t2):L|
Spend(csh, madgaon, t0)<=SpendBnd|M(csh, madgaon, t0)|
Spend(csh, net, t2)<=SpendBnd|60(csh, net, t2):U|
Spend(csh, cacoda, t0)==SpendBnd|300(csh, cacoda, t0)|
Spend(csh, ponje, t1)==SpendBnd|Th(csh, ponje, t1)|

-----------lnd Constraints -----------
Use(lnd, cacoda, t1)==UseBnd|DtSt(lnd, cacoda,

In [29]:
s.WF.dispositions

[(odp, WF, net, t0),
 (csh, WF, net, t0),
 (lnd, WF, net, t0),
 (gwp, WF, net, t0),
 (WF, net, t0),
 (LiR, WF, net, t0),
 (WF, net, t2)]

In [30]:
a = [1].insert(0, 0)

In [31]:
a

In [32]:
s.components

{'scopes': [hor, net],
 'assets': [csh, lnd],
 'gwp': gwp,
 'ap': ap,
 'epm': epm,
 'epf': epf,
 'ept': ept,
 'pocp': pocp,
 'odp': odp,
 'adpmn': adpmn,
 'adpmt': adpmt,
 'adpf': adpf,
 'wdp': wdp,
 'dm': dm,
 'market': market,
 'consumer': consumer,
 'earth': earth,
 'hor': hor,
 't0': t0,
 't1': t1,
 't2': t2,
 'net': net,
 'madgaon': madgaon,
 'ponje': ponje,
 'cacoda': cacoda,
 'alink': alink,
 'alink_': alink_,
 'blink': blink,
 'clink': clink,
 'clink_': clink_,
 'dlink': dlink,
 'csh': csh,
 'lnd': lnd,
 'H2': H2,
 'Solar': Solar,
 'Wind': Wind,
 'Power': Power,
 'Uranium': Uranium,
 'H2_L': H2_L,
 'CO2_AQoff': CO2_AQoff,
 'H2O': H2O,
 'CH4': CH4,
 'CO2': CO2,
 'CO2_Vent': CO2_Vent,
 'O2': O2,
 'CO2_DAC': CO2_DAC,
 'LiR': LiR,
 'WF': WF,
 'PV': PV,
 'SMRH': SMRH,
 'NGCC': NGCC,
 'SMR': SMR,
 'H2FC': H2FC,
 'DAC': DAC,
 'ASMR': ASMR,
 'AWE': AWE}

In [33]:
s.WF.data

Data|WF|

In [34]:
s.WF.constraints[0].__dict__

{'variable': Operate(WF, net, t2),
 'varbnd': <VarBnd.FREE: 5>,
 'parent': Capacity(WF, net, t2),
 'parameter': OpBnd|M(WF, net, t2):L|,
 'balance': None,
 'sumover': None,
 'multiply': True,
 'equality': 'leq',
 'disposition': (WF, net, t2),
 'name': 'BindUB[Operate(WF, net, t2)]',
 'equation': 'Operate(WF, net, t2)<=OpBnd|M(WF, net, t2):L|*Capacity(WF, net, t2)'}

In [35]:
s.constants

[5(csh, net, t2):L,
 60(csh, net, t2):U,
 300(csh, cacoda, t0),
 20(csh, lnd, net, t0),
 100(lnd, net, t2),
 0(H2, net, t0):L,
 4(H2, net, t0):U,
 5(csh, H2, net, t0):L,
 34(H2, net, t1):U,
 60(csh, H2, net, t0):U,
 300(csh, H2, cacoda, t0),
 0(Power, net, t1):L,
 0.3416(csh, Uranium, net, t0),
 20(CH4, net, t0):L,
 20(csh, CH4, net, t0),
 40(CH4, net, t0):U,
 20(O2, net, t0):L,
 1.484(gwp, LiR, net, t0),
 1.5(LiR, ponje, t0),
 1.5(LiR, WF, net, t0),
 30(lnd, WF, net, t0),
 50(gwp, WF, net, t0),
 70(csh, WF, net, t0):i,
 1000(WF, net, t0),
 2900(odp, WF, net, t0),
 4953(csh, WF, net, t0),
 202233(csh, WF, net, t0):i,
 1462000(csh, WF, net, t0),
 30(csh, Power, PV, net, t0),
 1333(csh, PV, net, t0, x(PV,capex,0)),
 1444(csh, PV, net, t0, x(PV,capex,1)),
 1555(csh, PV, net, t0, x(PV,capex,2)),
 3000(PV, net, t0, x(PV,capacity,0)),
 4000(PV, net, t0, x(PV,capacity,1)),
 5000(PV, net, t0, x(PV,capacity,2)),
 22623(csh, PV, net, t0),
 50(lnd, SMRH, net, t0),
 51.5(csh, SMRH, net, t0),
 1000

In [55]:
from energiapy._core._aliases._is_component import IsComponent

In [57]:
from enum import Enum, auto

In [58]:
class A(Enum):
    a = auto()
    b = auto()

In [59]:
@dataclass
class B:
    a: A

In [61]:
q = B(A.a)
r = B(A.b)
z = B(A.a)
w = B(A.b)
l = [q, r, z, w]

In [62]:
l_a = [i for i in l if i.a == A.a]

In [63]:
l_a

[B(a=<A.a: 1>), B(a=<A.a: 1>)]

In [36]:
s.PV.parameters

[CapBnd|3000(PV, net, t0, x(PV,capacity,0))|,
 CapBnd|4000(PV, net, t0, x(PV,capacity,1))|,
 CapBnd|5000(PV, net, t0, x(PV,capacity,2))|,
 CapExp|1333(csh, PV, net, t0, x(PV,capex,0))|,
 CapExp|1444(csh, PV, net, t0, x(PV,capex,1))|,
 CapExp|1555(csh, PV, net, t0, x(PV,capex,2))|,
 OpExp|22623(csh, PV, net, t0)|,
 ResCredit|DtSt(csh, Power, PV, net, t0)|,
 SellPrice|30(csh, Power, PV, net, t0)|]

In [37]:
s.PV.constraints

[Bind[Capacity(PV, net, t0, x(PV,capacity,0))],
 Bind[Capacity(PV, net, t0, x(PV,capacity,1))],
 Bind[Capacity(PV, net, t0, x(PV,capacity,2))],
 Calculate[Credit(csh, Power, PV, net, t0)],
 Calculate[ExpCap(csh, PV, net, t0, x(PV,capex,0))],
 Calculate[ExpCap(csh, PV, net, t0, x(PV,capex,1))],
 Calculate[ExpCap(csh, PV, net, t0, x(PV,capex,2))],
 Calculate[ExpOp(csh, PV, net, t0)],
 Calculate[ExpSell(csh, Power, PV, net, t0)]]

In [38]:
s.PV.variables[4].disposition.__dict__

{'ply': None,
 'emn': None,
 'csh': None,
 'res': None,
 'mat': None,
 'lnd': None,
 'pro': PV,
 'stg': None,
 'trn': None,
 'loc': None,
 'lnk': None,
 'ntw': net,
 'scl': t0,
 'mde': x(PV,capex,1),
 'index': (PV, net, t0, x(PV,capex,1)),
 'name': '(PV, net, t0, x(PV,capex,1))'}

In [46]:
from dataclasses import dataclass

In [53]:
@dataclass
class Ra:
    a: int = 0

    @classmethod
    def b(self, b):
        return self.a + b

In [54]:
Ra.b(2)

2

In [39]:
s.PV.capex[0].disposition.__dict__

{'ply': None,
 'emn': None,
 'csh': csh,
 'res': None,
 'mat': None,
 'lnd': None,
 'pro': PV,
 'stg': None,
 'trn': None,
 'loc': None,
 'lnk': None,
 'ntw': net,
 'scl': t0,
 'mde': x(PV,capex,0),
 'index': (csh, PV, net, t0, x(PV,capex,0)),
 'name': '(csh, PV, net, t0, x(PV,capex,0))'}

In [40]:
s.WF.variables

[Capacity(WF, net, t0),
 Capacity(WF, net, t2),
 EmitCap(gwp, WF, net, t0),
 EmitCap(odp, WF, net, t0),
 ExpCap(csh, WF, net, t0),
 ExpCapI(csh, WF, net, t0),
 ExpOp(csh, WF, net, t0),
 ExpOpI(csh, WF, net, t0),
 Operate(WF, net, t0),
 Operate(WF, net, t2),
 UseLnd(lnd, WF, net, t0),
 UseMat(LiR, WF, net, t0)]

In [40]:
s.WF.parameters

[CapBnd|1000(WF, net, t0)|,
 CapExpI|202233(csh, WF, net, t0):i|,
 CapExp|1462000(csh, WF, net, t0)|,
 LndUse|30(lnd, WF, net, t0)|,
 MatUse|1.5(LiR, WF, net, t0)|,
 OpBnd|DtSt(WF, net, t2)|,
 OpExpI|70(csh, WF, net, t0):i|,
 OpExp|4953(csh, WF, net, t0)|,
 OpnEmit|2900(odp, WF, net, t0)|,
 OpnEmit|50(gwp, WF, net, t0)|]

In [44]:
s.WF._consistent

True

In [45]:
s.WF.__dict__

{'basis': 'MW',
 'citation': None,
 'block': None,
 'introduce': None,
 'retire': None,
 'label': 'Wind mill array',
 'capacity': [1000(WF, net, t0)],
 'land': [30(lnd, WF, net, t0)],
 'material': [1.5(LiR, WF, net, t0)],
 'capex': [{202233(csh, WF, net, t0):i, 1462000(csh, WF, net, t0)}],
 'opex': [{70(csh, WF, net, t0):i, 4953(csh, WF, net, t0)}],
 'emission': [50(gwp, WF, net, t0), 2900(odp, WF, net, t0)],
 'buy': None,
 'sell': None,
 'buy_price': None,
 'sell_price': None,
 'credit': None,
 'penalty': None,
 'conversion': Conv(Power,WF),
 'produce': [True,
       b
  0    0
  1    1
  2    2
  3    3
  4    4
  5    5
  6    6
  7    7
  8    8
  9    9
  10  10
  11  11
  12  12
  13  13
  14  14
  15  15
  16  16
  17  17
  18  18
  19  19
  20  20
  21  21
  22  22
  23  23],
 'varying': None,
 'prod_max': None,
 'prod_min': None,
 'name': 'WF',
 '_named': True,
 '_model': :s:,
 '_consistent': True,
 'operate': [[M(WF, net, t2):L, DtSt(WF, net, t2):U]]}

In [42]:
s.PV.variables

[Capacity(PV, net, t0, x(PV,capacity,0)),
 Capacity(PV, net, t0, x(PV,capacity,1)),
 Capacity(PV, net, t0, x(PV,capacity,2)),
 Capacity(PV, net, t0, x(PV,capex,0)),
 Capacity(PV, net, t0, x(PV,capex,1)),
 Capacity(PV, net, t0, x(PV,capex,2)),
 Credit(csh, Power, PV, net, t0),
 ExpCap(csh, PV, net, t0, x(PV,capex,0)),
 ExpCap(csh, PV, net, t0, x(PV,capex,1)),
 ExpCap(csh, PV, net, t0, x(PV,capex,2)),
 ExpOp(csh, PV, net, t0),
 ExpSell(csh, Power, PV, net, t0),
 Operate(PV, net, t0),
 Sell(Power, PV, net, t0)]

In [43]:
s.variables

[Buy(CH4, NGCC, net, t0),
 Buy(CH4, net, t0),
 Buy(H2, H2FC, net, t0),
 Buy(H2O, NGCC, net, t0),
 Buy(H2O, net, t0),
 Buy(H2O, net, t2),
 Buy(Power, net, t1),
 Buy(Solar, net, t0),
 Buy(Uranium, ASMR, net, t0),
 Buy(Uranium, net, t0),
 Buy(Uranium, net, t2),
 Buy(Wind, net, t1),
 Capacity(ASMR, net, t0),
 Capacity(AWE, net, t0),
 Capacity(DAC, net, t0),
 Capacity(H2FC, net, t0),
 Capacity(NGCC, net, t0),
 Capacity(PV, net, t0, x(PV,capacity,0)),
 Capacity(PV, net, t0, x(PV,capacity,1)),
 Capacity(PV, net, t0, x(PV,capacity,2)),
 Capacity(PV, net, t0, x(PV,capex,0)),
 Capacity(PV, net, t0, x(PV,capex,1)),
 Capacity(PV, net, t0, x(PV,capex,2)),
 Capacity(SMR, net, t0),
 Capacity(SMRH, net, t0),
 Capacity(WF, net, t0),
 Capacity(WF, net, t2),
 Credit(csh, Power, PV, net, t0),
 EmitCap(gwp, WF, net, t0),
 EmitCap(odp, WF, net, t0),
 EmitUse(gwp, LiR, net, t0),
 ExpBuy(csh, CH4, NGCC, net, t0),
 ExpBuy(csh, CH4, net, t0),
 ExpBuy(csh, H2, H2FC, net, t0),
 ExpBuy(csh, H2O, NGCC, net, t0),
 E

## Declare processes

In [ ]:
# s.LiI = Process(store= 2000, conversion = {s.Power: 0.9}, store_loss = 0.2, store_cost= 0.4, capex = 3516428, fopex= 87910, vopex = 0, capacity= [1000,BigM], label='Lithium-ion battery', basis = 'MW')

## Declare locations

In [ ]:
PV.cap_max

In [ ]:
PV.factors

In [ ]:
houston.aspect

In [ ]:
houston.land_cost_factor

In [ ]:
# for i in houston.processes | sandiego.processes | newyork.processes:
# # for i in houston.resources | sandiego.resources | newyork.resources:
#     print(i, i.localizations)
#     print(i, i.factors)

In [ ]:
Power.ctype

## Declare transports

In [ ]:
# Train_H2 = Transport(name='Train_H2', resources={
#                      H2}, trans_max=bigM, trans_loss=0.03, trans_cost=1.667*10**(-3), label='Railway for hydrogen transportation')
# Grid = Transport(name='Grid', resources={
#                  Power}, trans_max=bigM, trans_loss=0.001, trans_cost=0.5*10**(-3), label='Railroad transport')


Train_H2 = Transport(
    name='Train_H2',
    resources={H2},
    cap_max=BigM,
    trans_loss=0.03,
    capex=1000,
    vopex=1.667,
    fopex=(0, 1),
    acid=34,
    eutf=55,
    label='Railway for hydrogen transportation',
)
Grid = Transport(
    name='Grid',
    resources={H2, Power},
    cap_max=BigM,
    trans_loss=0.001,
    capex=500,
    vopex=0.444,
    label='Railroad transport',
)

In [ ]:
Train_H2.cap_max

## Declare Network

In [ ]:
distance_matrix = [[2366], [2620]]

transport_matrix = [
    [[Train_H2]],
    [[Grid]],
]
capacity_factor = {
    (houston, newyork): {Train_H2: weather_sd[['wind_speed']]},
    (sandiego, newyork): {Train_H2: weather_sd[['wind_speed']]},
}

network = Network(
    name='network',
    scales=scales,
    land_max=(0, 5),
    land_max_factor=DataFrame({'a': [5]}),
    capacity_factor=capacity_factor,
    vopex_factor={(houston, sandiego): {Train_H2: weather_sd[['wind_speed']]}},
    sources=[houston, sandiego],
    sinks=[newyork],
    distance_matrix=distance_matrix,
    transport_matrix=transport_matrix,
)

In [ ]:
scenario = Scenario(
    name='scenario_full', network=network, scales=scales, label='full_case'
)

In [ ]:
scenario

In [ ]:
for i in houston.resources:
    print(i.ctype)

In [ ]:
houston.resources_produce

In [ ]:
ResourceType.all()

In [ ]:
ResourceType.resource_level()

In [ ]:
ResourceType.location_level()

In [ ]:
ResourceType.transport_level()

In [ ]:
ProcessType.all()

In [ ]:
ProcessType.process_level()

In [ ]:
ProcessType.location_level()

In [ ]:
LocationType.location_level()

In [ ]:
Power.transport

In [ ]:
Solar.availability

In [ ]:
Power.demand

In [ ]:
[((houston, sandiego), CO2), ((houston, sandiego), H2), ((houston, newyork), CO2)]

In [ ]:
comp_loc_dict = scenario.resources_locations_discharge

In [ ]:
scenario.locations_land

In [ ]:
scenario.locations_locations_land

In [ ]:
comp_loc_ordered_set = [(i, j)
                        for i in comp_loc_dict for j in comp_loc_dict[i]]

In [ ]:
scenario.resources_locations_store

In [ ]:
comp_loc_ordered_set

In [ ]:
comp_loc_ordered_set

In [ ]:
scenario.resources_locations_discharge

In [ ]:
scenario.resources_discharge

In [ ]:
scenario.processes_locations_expenditure

In [ ]:
resources_consume_locations = {
    resource: set() for resource in scenario.resources_consume
}
for resource in scenario.resources_consume:
    for location in scenario.locations:
        if hasattr(location, 'resources_consume'):
            if resource in location.resources_consume:
                resources_consume_locations[resource].add(location)

In [ ]:
resources_consume_locations = {
    resource: {
        location
        for location in scenario.locations
        if hasattr(location, 'resources_consume')
        and resource in location.resources_consume
    }
    for resource in scenario.resources_consume
}

In [ ]:
scenario.resources_consume

In [ ]:
resources_consume_locations = {
    resource: {
        location
        for location in self.locations
        if hasattr(location, 'resources_consume')
        and resource in location.resources_consume
    }
    for resource in self.resources_consume
}

In [ ]:
resources_consume_locations

In [ ]:
resources_demand_locations = {
    resource: {
        location
        for location in scenario.locations
        if hasattr(location, 'resources_demand')
        and resource in location.resources_demand
    }
    for resource in scenario.resources_demand
}

In [ ]:
scenario.resource_classifications()

In [ ]:
{
    component: {
        location
        for location in getattr(self, 'locations')
        if hasattr(location, f'{component_set}_{parameter.lower()}')
        and resource in getattr(location, f'{component_set}_{parameter.lower()}')
    }
    for resource in getattr(self, f'{component_set}_{parameter.lower()}')
}

In [ ]:
scenario.process_classifications()

In [ ]:
[
    ((houston.name, sandiego.name), CO2.name),
    ((houston.name, sandiego.name), H2.name),
    ((houston.name, newyork.name), CO2.name),
]

In [ ]:
help(ResourceParamType)

In [ ]:
from pyomo.environ import *

# from energiapy.utils.scale_utils import scale_pyomo_set

In [ ]:
scales1 = TemporalScale(discretization_list=[
                        1, 4], design_scale=0, scheduling_scale=1)

In [ ]:
type(scales1)

In [ ]:
scales2 = TemporalScale(
    discretization_list=[2, 2, 2], design_scale=0, scheduling_scale=2
)

In [ ]:
scales1.index_list

In [ ]:
scales2.index_list

In [ ]:
m = ConcreteModel()
m.scales = Set(scales1.list, initialize=scales1.scale)
m.scales_scheduling = scale_pyomo_set(
    instance=m, scale_level=scales1.scheduling_scale)
m.scales_design = scale_pyomo_set(instance=m, scale_level=scales1.design_scale)

In [ ]:
scales.scheduling_scale

In [ ]:
# m.a = Set(initialize = [(i, j) for i in resources_consume_locations for j in resources_consume_locations[i]])
# m.b = Set(initialize = [(i.name, j.name) for i in scenario.resources_locations_consume for j in scenario.resources_locations_consume[i]])
# m.c = Set(initialize = [((houston.name, sandiego.name), CO2.name), ((houston.name, sandiego.name), H2.name), ((houston.name, newyork.name), CO2.name)])
# m.A = Var(m.a, m.scales_scheduling, within = NonNegativeReals)
# m.B = Var(m.b, m.scales_design, within = NonNegativeReals)
# m.C = Var(m.c, m.scales_scheduling, within = NonNegativeReals)

In [ ]:
from pyomo.environ import *

m = ConcreteModel()

In [ ]:
m.d = Set(
    initialize=[
        (Train_H2.name, (houston.name, sandiego.name)),
        (Grid.name, (houston.name, sandiego.name)),
        (Grid.name, (houston.name, newyork.name)),
    ]
)
m.D = Var(m.d, m.scales_design, within=NonNegativeReals)

In [ ]:
m.f = Set(initialize=[Train_H2.name, Grid.name])
m.F = Var(m.f, m.scales_design, within=NonNegativeReals)

In [ ]:
m.g = Set(
    initialize=[
        (CO2.name, (houston.name, sandiego.name)),
        (Power.name, (houston.name, sandiego.name)),
        (H2.name, (houston.name, newyork.name)),
    ]
)

In [ ]:
m.G = Var(m.g, m.scales_scheduling, within=NonNegativeReals)

In [ ]:
m.G.pprint()

In [ ]:
m = ConcreteModel()

In [ ]:
import gurobipy

m = gurobipy.read('example.lp')

In [ ]:
import pandas

In [ ]:
a = DataFrame(data={'a': [1, 2, 3]})

In [ ]:
b = 

In [ ]:
class LPFileCreator:
    def __init__(self, objective, constraints, bounds, file_name="problem.lp"):
        self.objective = objective
        self.constraints = constraints
        self.bounds = bounds
        self.file_name = file_name

    def create_lp_file(self):
        content = self._generate_lp_content()
        with open(self.file_name, "w") as file:
            file.write(content)

    def _generate_lp_content(self):
        lp_content = []

        # Add Objective Function
        lp_content.append("\\* Problem Statement *\\")
        lp_content.append("\nMaximize")
        lp_content.append(f" obj: {self.objective}\n")

        # Add Constraints
        lp_content.append("Subject To")
        for i, constraint in enumerate(self.constraints, start=1):
            lp_content.append(f" c{i}: {constraint}")
        lp_content.append("")

        # Add Bounds
        lp_content.append("Bounds")
        for bound in self.bounds:
            lp_content.append(f" {bound}")
        lp_content.append("")

        # Add End
        lp_content.append("End")

        return "\n".join(lp_content)

In [ ]:
a = [7, 2, 3].sort()

In [ ]:
a

In [ ]:
sorted(a)

In [ ]:
houston.resources_consume

In [ ]:
[f'cons_{i.name}_' for i in houston.resources_consume]

In [ ]:
def tuples_to_strings(tuple_list):
    return ['_'.join(map(str, t)) for t in tuple_list]

In [ ]:
scales = TemporalScale([1, 4], design_scale=0, scheduling_scale=1)

In [ ]:
var = ['cons']  # , 'sell']

In [ ]:
res_list = ['res1', 'res2', 'res3']

In [ ]:
time_index = tuples_to_strings(scales.index_list[1])

In [ ]:
value = {'res1': 5, 'res2': 7, 'res3': 8}

In [ ]:
cons = list()

In [ ]:
for i, j, k in product(var, res_list, time_index):
    cons.append(f'{i}_{j}_{k} <= {value[j]}')

In [ ]:
ooo = {'res1': 7, 'res2': 5, 'res3': 9}

In [ ]:
k = [f'{ooo[i]} {i}' for i in res_list]

In [ ]:
obj = ' + '.join(k)
obj

In [ ]:
for i,j,k in product(var, res_list, time_index):


In [ ]:
bou = [f'0 <= {i}' for i in var]

In [ ]:
# Define the problem components
objective = "x + 2 y"
constraints = ["x + y >= 1", "x <= 2", "y <= 2"]
bounds = ["0 <= x", "0 <= y"]

# Create the LP file
# creator = LPFileCreator(objective, constraints, bounds, file_name="example.lp")

creator = LPFileCreator(obj, cons, bou, file_name="example.lp")
creator.create_lp_file()
print("LP file 'example.lp' created successfully.")

In [ ]:
import gurobipy as gb

a = gb.read('example.lp')

In [ ]:
a.optimize()

In [ ]:
Maximize
  x + y + z
Subject To
  c0: x + y = 1
  c1: x + 5 y + 2 z <= 10
  qc0: x + y + [ x ^ 2 - 2 x * y + 3 y ^ 2 ] <= 5
Bounds
  0 <= x <= 5
  z >= 2
Generals
  x y z
End


In [ ]:
H2O.purchase_cost

In [ ]:
H2O.cons_max

In [ ]:
m.optimize()

In [ ]:
m.getVars()

In [ ]:
plot_scenario.capacity_factor(
    scenario=scenario, location=houston, process=PV, fig_size=(9, 5), color='orange'
)

![cap fac pv](plots / cf_ho.png)

In [ ]:
plot_scenario.capacity_factor(
    scenario=scenario, location=sandiego, process=WF, fig_size=(9, 5), color='blue'
)

![cap fac wf](plots / wf_sd.png)

In [ ]:
plot_scenario.demand_factor(
    scenario=scenario, location=newyork, resource=Power, fig_size=(9, 5), color='red'
)

![df ny](plots / df_ny.png)

## Optimize for minimum cost

In [ ]:
milp_cost = formulate(
    scenario=scenario,
    constraints={
        Constraints.COST,
        Constraints.INVENTORY,
        Constraints.PRODUCTION,
        Constraints.RESOURCE_BALANCE,
        Constraints.TRANSPORT,
        Constraints.MODE,
        Constraints.NETWORK,
    },
    objective=Objective.COST,
)

**Ensure no discharge of power in Houston and San Diego**

While the demand for power is zero in these locations, Power is still a dischargeable resource. Hence, the discharge bounds need to be set to zero.
If not set to zero, the system chooses to dispense power instead of establishing storage networks.

In [ ]:
milp_cost.constraint_specific_location_discharge_ho = (
    constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location=houston
    )
)
milp_cost.constraint_specific_location_discharge_sd = (
    constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location=sandiego
    )
)

In [ ]:
results_cost = solve(
    scenario=scenario,
    instance=milp_cost,
    solver='gurobi',
    name="results_cost",
    print_solversteps=True,
)

**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(
    results=results_cost, source='SD', sink='NY', resource='H2', transport='Train_H2'
)

![sch t sdny](plots / sch_sdny.png)

In [ ]:
plot_results.transport(
    results=results_cost, source='HO', sink='NY', resource='H2', transport='Train_H2'
)

![sch hony](plots / sch_hony.png)

**Cost contribution by technology**

In [ ]:
plot_results.cost(
    results=results_cost,
    x=CostX.PROCESS_WISE,
    y=CostY.CAPEX,
    location='HO',
    fig_size=(8, 6),
)
plot_results.cost(
    results=results_cost,
    x=CostX.PROCESS_WISE,
    y=CostY.VOPEX,
    location='HO',
    fig_size=(8, 6),
)
plot_results.cost(
    results=results_cost,
    x=CostX.PROCESS_WISE,
    y=CostY.FOPEX,
    location='HO',
    fig_size=(8, 6),
)
plot_results.cost(
    results=results_cost,
    x=CostX.PROCESS_WISE,
    y=CostY.TOTAL,
    location='HO',
    fig_size=(8, 6),
)

![total cost ho](plots / total_ho.png)

**Location-wise cost contribution by technology**

In [ ]:
plot_results.cost(
    results=results_cost, x=CostX.LOCATION_WISE, y=CostY.CAPEX, fig_size=(8, 6)
)
plot_results.cost(
    results=results_cost, x=CostX.LOCATION_WISE, y=CostY.VOPEX, fig_size=(8, 6)
)
plot_results.cost(
    results=results_cost, x=CostX.LOCATION_WISE, y=CostY.FOPEX, fig_size=(8, 6)
)
plot_results.cost(
    results=results_cost, x=CostX.LOCATION_WISE, y=CostY.TOTAL, fig_size=(8, 6)
)

![total lw](plots / total_lw.png)

**Inventory and production schedules**

In [ ]:
plot_results.schedule(
    results=results_cost,
    y_axis='Inv',
    component='LiI_Power_stored',
    location='SD',
    fig_size=(9, 5),
    color='steelblue',
)

![inv lii](plots / lii_inv.png)

In [ ]:
plot_results.schedule(
    results=results_cost,
    y_axis='P',
    component='PV',
    location='SD',
    fig_size=(9, 5),
    color='steelblue',
)

![pv p](plots / pv_p.png)

## Optimize to maximize resource discharge 

In [ ]:
milp_demand = formulate(
    scenario=scenario,
    constraints={
        Constraints.COST,
        Constraints.INVENTORY,
        Constraints.PRODUCTION,
        Constraints.RESOURCE_BALANCE,
        Constraints.TRANSPORT,
        Constraints.MODE,
        Constraints.NETWORK,
    },
    objective=Objective.MAX_DISCHARGE,
    objective_resource=Power,
)

In [ ]:
milp_demand.constraint_specific_location_discharge_ho = (
    constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location=houston
    )
)
milp_demand.constraint_specific_location_discharge_sd = (
    constraint_specific_location_discharge(
        instance=milp_demand,
        network_scale_level=0,
        bounds={Power: 0},
        location=sandiego,
    )
)

In [ ]:
results_demand = solve(
    scenario=scenario,
    instance=milp_demand,
    solver='gurobi',
    name="results_demand",
    print_solversteps=True,
)

**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(
    results=results_demand, source='SD', sink='NY', resource='H2', transport='Train_H2'
)

![sch t sdny2](plots / sch_sdny2.png)

In [ ]:
plot_results.transport(
    results=results_demand, source='HO', sink='NY', resource='H2', transport='Train_H2'
)

![sch t hony](plots / sch_hony2.png)